In [1]:
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWtgzh4zMaa3v93O1DW_C0hJtbb1gtu0wz28q8xGyCTSf6LhvpjR6DDtkt8

Successfully saved authorization token.


In [3]:
def get_dates(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    dates = []
    while start + timedelta(days=1) <= end:
        dates.append([start.strftime('%Y-%m-%d'), (start + timedelta(days=1)).strftime('%Y-%m-%d')])
        start += timedelta(days=1)
    return dates

In [4]:
def get_data(start_date, end_date, roi):
    # sentinel 2 dataset
    dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
        .mean()
    # landsat 8 dataset
    dataset2 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .median()
    # dict to keep track of null vals
    is_null = {
        'ndwi': False,
        'mndwi': False,
        'ndci': False,
        'ndti': False,
        'do': False,
        'ndsi': False,
        'ph': False,
        'chl_a': False,
        'ssc': False,
        'wst': False,
    }
    # ndwi
    try:
        ndwi_bands = ['B3', 'B5']
        ndwi = dataset.normalizedDifference(ndwi_bands).rename('ndwi')
        latlon = ee.Image.pixelLonLat().addBands(ndwi)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndwi = np.array((ee.Array(latlon.get('ndwi')).getInfo()))
    except:
        is_null["ndwi"] = True

    # mndwi
    try:
        band_mndwi = ['B3', 'B6']
        mndwi = dataset.normalizedDifference(band_mndwi).rename('mndwi')
        latlon = ee.Image.pixelLonLat().addBands(mndwi)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_mndwi = np.array((ee.Array(latlon.get('mndwi')).getInfo()))
    except:
        is_null["mndwi"] = True

    # ndci
    try:
        band_ndci = ['B4', 'B3']
        ndci = dataset.normalizedDifference(band_ndci).rename('ndci')
        latlon = ee.Image.pixelLonLat().addBands(ndci)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndci = np.array((ee.Array(latlon.get('ndci')).getInfo()))
    except:
        is_null["ndci"] = True
    
    # do
    try:
        band_do = ['B8', 'B11']
        dissolvedoxygen = dataset.normalizedDifference(band_do).rename('do')
        latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=geometry,
            scale=8)
        data_do = np.array((ee.Array(latlon.get('do')).getInfo()))
    except: 
        is_null["do"] = True
    # ndti
    try:    
        band_ndti = ['B5', 'B11']
        ndti = dataset.normalizedDifference(band_ndti).rename('ndti')
        latlon = ee.Image.pixelLonLat().addBands(ndti)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndti = np.array((ee.Array(latlon.get('ndti')).getInfo()))
    except:
        is_null["ndti"] = True
        
    # ndsi
    try:
        band_ndsi = ['B3', 'B11']
        ndsi = dataset.normalizedDifference(band_ndsi).rename('ndsi')
        latlon = ee.Image.pixelLonLat().addBands(ndsi)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndsi = np.array((ee.Array(latlon.get('ndsi')).getInfo()))
    except:
        is_null["ndsi"] = True
    
    # ph
    try:
        ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(
            dataset.select('B1').divide(dataset.select('B8')))).rename('ph')
        latlon = ee.Image.pixelLonLat().addBands(ph)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ph = np.array((ee.Array(latlon.get('ph')).getInfo()))
    except:
        is_null["ph"] = True

    # chlorophyll a
    try:
        Rrs_red = dataset.select('B3').divide(0.1)
        Rrs_blue = dataset.select('B2').divide(0.1)
        chl_a = Rrs_red.divide(Rrs_blue).pow(2.72).multiply(10.8).rename('chl_a')
        latlon = ee.Image.pixelLonLat().addBands(chl_a)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_chl_a = np.array((ee.Array(latlon.get('chl_a')).getInfo()))
    except:
        is_null["chl_a"] = True

    # suspended sediment conc
    try:
        ssc = ee.Image(0.0113).multiply(ndwi.pow(3)).subtract(ee.Image(0.0135).multiply(
            ndwi.pow(2))).add(ee.Image(0.0075).multiply(ndwi)).add(ee.Image(2.5823)).rename('ssc')
        latlon = ee.Image.pixelLonLat().addBands(ssc)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ssc = np.array((ee.Array(latlon.get('ssc')).getInfo()))
    except:
        is_null["ssc"] = True

    # WST
    try:
        tir1 = dataset.select('B10')  # 10.8-11.3 µm
        tir2 = dataset.select('B11')  # 11.5-12.5 µm
        #kelvin = tir2.divide(10).add(273.15)
        #wst = kelvin.multiply(1.0158).subtract(
        #    tir1.multiply(0.9543)).subtract(2.7886).rename('wst')
        #latlon = ee.Image.pixelLonLat().addBands(wst)
        #latlon = latlon.reduceRegion(
        #    reducer=ee.Reducer.toList(),
        #    geometry=roi,
        #    scale=8)
        #data_wst = np.array((ee.Array(latlon.get('wst')).getInfo()))
        kelvin = tir2.divide(10).add(273.15)
        lst = kelvin.divide(ee.Image(1).toFloat().divide(tir2.divide(14380).add(1).log()))
        emissivity = ee.Image(0.98)
        atm_corr = lst.multiply(0.99).add(0.11).multiply(emissivity).subtract(2.5)
        wst = atm_corr.subtract(273.15).rename('wst')
        latlon = ee.Image.pixelLonLat().addBands(wst)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_wst = np.array((ee.Array(latlon.get('wst')).getInfo()))
    except:
        is_null["wst"] = True

    # adding data one by one
    our_data = {}

    if is_null["ndwi"]:
        our_data["ndwi"] = None
    else:
        our_data["mndwi"] = data_ndwi
    
    if is_null["mndwi"]:
        our_data["ndwi"] = None
    else:
        our_data["ndwi"] = data_mndwi
    
    if is_null["ndci"]:
        our_data["ndci"] = None
    else:
        our_data["ndci"] = data_ndci
    
    if is_null["ndti"]:
        our_data["ndti"] = None
    else:
        our_data["ndti"] = data_ndti
    
    if is_null["do"]:
        our_data["do"] = None
    else:
        our_data["do"] = data_do
    
    if is_null["ndsi"]:
        our_data["ndsi"] = None
    else:
        our_data["ndsi"] = data_ndsi
    
    if is_null["ph"]:
        our_data["ph"] = None
    else:
        our_data["ph"] = data_ph
    
    if is_null["chl_a"]:
        our_data["chl_a"] = None
    else:
        our_data["chl_a"] = data_chl_a
    
    if is_null["ssc"]:
        our_data["ssc"] = None
    else:
        our_data["ssc"] = data_ssc
    
    if is_null["wst"]:
        our_data["wst"] = None
    else:
        our_data["wst"] = data_wst

    dataframe = pd.DataFrame(our_data)
    
    return dataframe

In [5]:
def get_dataframe(start_date, end_date, polygon):
    df_data = pd.DataFrame()
    df_data["ndwi"] = []
    df_data["mndwi"] = []
    df_data["ndci"] = []
    df_data["ndti"] = []
    df_data["do"] = []
    df_data["ndsi"] = []
    df_data["ph"] = []
    df_data["chl_a"] = []
    df_data["ssc"] = []
    df_data["wst"] = []
    df_data["date"] = []

    dates = get_dates(start_date, end_date)

    for date in dates:
        try:
            df = get_data(date[0], date[1], polygon)
            means = {}
            for column in df.columns:
                try:
                    means[column] = [df[column].mean()]
                except:
                    means[column] = None
            means["date"] = [date[0]]
            mean_data = pd.DataFrame(means)
            df_data = pd.concat([df_data, mean_data])
        except:
            mean_data = {}
            mean_data["date"] = date[0]
            mean_data["ndwi"] = [None]
            mean_data["mndwi"] = [None]
            mean_data["ndci"] = [None]
            mean_data["ndti"] = [None]
            mean_data["do"] = [None]
            mean_data["ndsi"] = [None]
            mean_data["ph"] = [None]
            mean_data["chl_a"] = [None]
            mean_data["ssc"] = [None]
            mean_data["wst"] = [None]
            final_mean = pd.DataFrame(mean_data)
            
            df_data = pd.concat([df_data, final_mean])
    
    return df_data

In [6]:
geometry = ee.Geometry.Polygon([
    [[77.38201496321473, 23.20425936091927], 
    [77.380083772724, 23.20240549169144], 
    [77.38107082564149, 23.20177438140686], 
    [77.3814141483954, 23.20082771039299], 
    [77.38244411665711, 23.200985489360875], 
    [77.38313076216492, 23.201656047896766], 
    [77.38441822249207, 23.201695492411773], 
    [77.38463279921326, 23.200867155152423], 
    [77.38570568281922, 23.19790876589711], 
    [77.38617775160584, 23.19637037761234], 
    [77.3861348362616, 23.1950292041526], 
    [77.38514778334412, 23.19487141815525], 
    [77.38317367750916, 23.19554200735805], 
    [77.38214370924744, 23.195068650622837], 
    [77.38025543410096, 23.193096312855694], 
    [77.38222953993592, 23.192386264137998], 
    [77.38544819075379, 23.193214653942096], 
    [77.38540527540955, 23.192228475022077], 
    [77.38420364577088, 23.190847812316374], 
    [77.38407489973817, 23.189269894628183], 
    [77.38596317488465, 23.191281736416926], 
    [77.38686439711365, 23.191478974179027], 
    [77.38617775160584, 23.19293852457757], 
    [77.38699314314637, 23.193490782736447], 
    [77.38802311140809, 23.194358612378032], 
    [77.38892433363709, 23.194358612378032], 
    [77.38883850294862, 23.193411888853383], 
    [77.39046928602967, 23.192977971664554], 
    [77.39004013258729, 23.192346816876473], 
    [77.39068386275086, 23.191202841230634], 
    [77.3933016987494, 23.19226792231852], 
    [77.39634868819032, 23.192544053067735], 
    [77.39750740248475, 23.193451335800724], 
    [77.39815113264832, 23.193806357803222], 
    [77.39866611677918, 23.19376691096059], 
    [77.40029689986024, 23.193845804634197], 
    [77.40192768294129, 23.194200825589387], 
    [77.40192768294129, 23.19471363197173], 
    [77.40312931257996, 23.1949503111772],
    [77.40347263533387, 23.194595292211893],
    [77.40398761946473, 23.19471363197173],
    [77.40441677290711, 23.19400359184176],
    [77.40557548720155, 23.194240272304008],
    [77.40561840254578, 23.193411888853383],
    [77.40690586287293, 23.19329354794152],
    [77.4095666142157, 23.202011048112766],
    [77.40857956129823, 23.201892714812193],
    [77.40694877821717, 23.20205049252301],
    [77.40656254011903, 23.202563268796972],
    [77.40639087874207, 23.20350992751922],
    [77.40707752424989, 23.203904366674646],
    [77.40643379408631, 23.204298804666024],
    [77.40626213270936, 23.20473268511204],
    [77.40604755598817, 23.204456579536636],
    [77.4051892491034, 23.20473268511204],
    [77.40471718031678, 23.20504823364264],
    [77.40454551893983, 23.204062142010887],
    [77.40471718031678, 23.20358881544343],
    [77.40433094221864, 23.20323381941782],
    [77.40471718031678, 23.202957710746055],
    [77.40420219618592, 23.20082771039299],
    [77.40321514326844, 23.200867155152423],
    [77.40265724379334, 23.20122215746353],
    [77.40218517500672, 23.20090659990022],
    [77.40128395277772, 23.201458825147164],
    [77.40051147658143, 23.20177438140686],
    [77.39995357710633, 23.202721045716288],
    [77.39930984694276, 23.20177438140686],
    [77.39759323317323, 23.20082771039299],
    [77.39772197920594, 23.200354372371955],
    [77.39707824904237, 23.199959922740792],
    [77.39634868819032, 23.198461003527797],
    [77.39514705855164, 23.19763264622637],
    [77.39416000563416, 23.1977904289652],
    [77.39416000563416, 23.198105993884226],
    [77.39377376753602, 23.19822433053681],
    [77.3942029209784, 23.198539894431676],
    [77.39394542891297, 23.19901323887734],
    [77.39433166701112, 23.199171019986796],
    [77.39583370405946, 23.20122215746353],
    [77.39652034956727, 23.202011048112766],
    [77.39652034956727, 23.20358881544343],
    [77.3958766194037, 23.204298804666024],
    [77.39484665114198, 23.20481157231454],
    [77.39343044478211, 23.20473268511204],
    [77.3930871220282, 23.205008790117045],
    [77.39235756117615, 23.20481157231454],
    [77.39154216963563, 23.205008790117045],
    [77.39068386275086, 23.204535466902104],
    [77.39068386275086, 23.20335215153112],
    [77.39068386275086, 23.202760489917026],
    [77.39016887862, 23.202563268796972],
    [77.38866684157166, 23.20122215746353],
    [77.38746521193299, 23.20130104673797],
    [77.38338825423035, 23.20350992751922],
    [77.38304493147645, 23.203628259388076],
    [77.38248703200135, 23.204377692124613],
    [77.38188621718201, 23.204535466902104],
    [77.38201496321473, 23.20425936091927]]])

In [47]:
start = '2020-10-01'
end = '2020-12-01'

In [ ]:
final_data = get_dataframe(start, end, geometry)

In [ ]:
final_data

In [ ]:
final_data.to_csv(f'{start}.csv')